<a href="https://colab.research.google.com/github/luca-arts/seeingtheimperceptible/blob/main/notebooks/syntheticPoseImageGeneration/tests/Pose_with_Style.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pose with Style: Detail-Preserving Pose-Guided Image Synthesis with Conditional StyleGAN

[GitHub](https://github.com/BadourAlBahar/pose-with-style)

## 1. Preparations

Before start, make sure that you choose

Runtime Type = Python 3
Hardware Accelerator = GPU

In [1]:
!nvidia-smi

Mon Jul 18 09:38:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 2. linking nextcloud

Connecting to the external NextCloud drive 

In [2]:
# we'll link the dataset from next-cloud
!curl https://raw.githubusercontent.com/luca-arts/seeingtheimperceptible/main/notebooks/database_mod.py -o /content/database_mod.py

from database_mod import *

link_nextcloud()

nextcloud = '/content/database/'

input_folder, output_folder = create_io(database=nextcloud,topic='syntheticPoseImageGeneration',library='PoseStyle')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2510  100  2510    0     0  12550      0 --:--:-- --:--:-- --:--:-- 12550
what's the username for nextcloud? colab
what's the password for user colab? ··········
0
Please enter the username to authenticate with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Username: Please enter the password to authenticate user colab with server
https://cloud.bxlab.net/remote.php/dav/files/colab/colabfiles/ or hit enter for none.
  Password:  


## 3. clone GIT repo

In [3]:
import os
root_path = '/content/PoseStyle'

# clone the repository
if not os.path.exists(root_path):
  !git clone https://github.com/BadourAlBahar/pose-with-style {root_path}

%ls

Cloning into '/content/PoseStyle'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 43 (delta 0), reused 0 (delta 0), pack-reused 38
Unpacking objects: 100% (43/43), done.
database/  database_mod.py  PoseStyle/  __pycache__/  sample_data/


## 4. Setting up the environment

In [4]:
#don't know if we need to install this - if i run it the code gives me an error
#!pip install pytorch==1.9.0 torchvision==0.10.0 torchaudio==0.9.0 cudatoolkit=10.2 -c pytorch

req_path = root_path + '/requirements.txt'

# Set up the environment
print('\n> Install dependencies')

print('\n> Install pytorch')
!pip install pytorch==1.9.0 --quiet 

print('\n> Install torchvision')
!pip install torchvision==0.10.0 --quiet 

print('\n> Install torchaudio')
!pip install torchaudio==0.9.0 --quiet

print('\n> Install cudatoolkit')
!pip install cudatoolkit=10.2 -c pytorch --quiet

print('\n> Install requirements.txt')
!pip install -r {req_path} --quiet

print('\n> Install OpenCV')
!pip install opencv-python


> Install dependencies

> Install pytorch
ERROR: Could not find a version that satisfies the requirement pytorch==1.9.0 (from versions: 0.1.2, 1.0.2)
ERROR: No matching distribution found for pytorch==1.9.0

> Install torchvision
     |████████████████████████████████| 22.1 MB 98.2 MB/s 
     |████████████████████████████████| 831.4 MB 2.8 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.0 requires torch==1.12.0, but you have torch 1.9.0 which is incompatible.
torchaudio 0.12.0+cu113 requires torch==1.12.0, but you have torch 1.9.0 which is incompatible.

> Install torchaudio
     |████████████████████████████████| 1.9 MB 5.4 MB/s 

> Install cudatoolkit
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'

> Install requirements.txt

> Install OpenCV
Looking in indexes: https://pypi.org/simple, https://us

In [5]:
import torch
print("pytorch ::: " + torch.__version__)

pytorch ::: 1.9.0+cu102


copy UV space to util folder

In [6]:
import shutil

src_path = "/content/database/syntheticPoseImageGeneration/PoseStyle/util/dp_uv_lookup_256.npy"
dst_path = "/content/PoseStyle/util/dp_uv_lookup_256.npy"

shutil.copy(src_path, dst_path)

'/content/PoseStyle/util/dp_uv_lookup_256.npy'

## 5. Reposing Inference

In [9]:
%cd {root_path}
%ls

/content/PoseStyle
data/       distributed.py*      LICENSE    README.md*        test.py*
DATASET.md  garment_transfer.py  model.py*  requirements.txt  train.py
dataset.py  inference.py         op/        sphereface.py     util/


In [12]:
CCM_path = "/content/database/syntheticPoseImageGeneration/PoseStyle/pretrained_models/CCM_epoch50.pt"
model_path = "/content/database/syntheticPoseImageGeneration/PoseStyle/pretrained_models/posewithstyle.pt"

!python inference.py --input_path /content/PoseStyle/data --CCM_pretrained_model {CCM_path} --pretrained_model {model_path}

Traceback (most recent call last):
  File "inference.py", line 12, in <module>
    from model import Generator
  File "/content/PoseStyle/model.py", line 11, in <module>
    from op import FusedLeakyReLU, fused_leaky_relu, upfirdn2d, conv2d_gradfix
  File "/content/PoseStyle/op/__init__.py", line 1, in <module>
    from .fused_act import FusedLeakyReLU, fused_leaky_relu
  File "/content/PoseStyle/op/fused_act.py", line 15, in <module>
    os.path.join(module_path, "fused_bias_act_kernel.cu"),
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py", line 1092, in load
    keep_intermediates=keep_intermediates)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py", line 1303, in _jit_compile
    is_standalone=is_standalone)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py", line 1374, in _write_ninja_file_and_build_library
    verify_ninja_availability()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_e